# Imports

In [ ]:
import pandas as pd 
import numpy as np

### Base com atributos categóricos

In [ ]:
# Base tirada de https://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame

# Vitória para quando X for o primeiro

jogo_da_velha = pd.read_csv('databases/tic-tac-toe.csv',
                      header=None, 
                      names=[
                          "top-left",
                          "top-middle",
                          "top-right",
                          "middle-left",
                          "middle-middle",
                          "middle-right",
                          "bottom-left",
                          "bottom-middle", 
                          "bottom-right",
                          "Class"
                      ])

jogo_da_velha

In [ ]:
# jogo_da_velha.describe()

In [ ]:
# jogo_da_velha.dtypes

## Pré-Processamento Inicial

In [ ]:
atributos_categoricos = [
    "top-left",
    "top-middle",
    "top-right",
    "middle-left",
    "middle-middle",
    "middle-right",
    "bottom-left",
    "bottom-middle", 
    "bottom-right"
    ]

jogo_da_velha = pd.get_dummies(jogo_da_velha, 
                         columns=atributos_categoricos)
jogo_da_velha

In [ ]:
# from sklearn.preprocessing import OrdinalEncoder
# atributos = [
#                           "top-left",
#                           "top-middle",
#                           "top-right",
#                           "middle-left",
#                           "middle-middle",
#                           "middle-right",
#                           "bottom-left",
#                           "bottom-middle", 
#                           "bottom-right"
#                       ]
# enc = OrdinalEncoder()
# enc.fit(jogo_da_velha[atributos])
# jogo_da_velha[atributos] = enc.transform(jogo_da_velha[atributos])
# jogo_da_velha = jogo_da_velha.replace({"positive": "V", "negative": "F"})
# jogo_da_velha

In [ ]:
from sklearn.preprocessing import StandardScaler

# jogo_da_velha[atributos] = StandardScaler().fit_transform(jogo_da_velha[atributos])
# #jogo_da_velha = pd.DataFrame(jogo_da_velha)

# jogo_da_velha


atributos = jogo_da_velha.columns.values.tolist()
# if not reordered:
#     for i in range(2):
#         e = atributos.pop(0)
#         atributos.append(e)
#     atributos

# reordered = True
# jogo_da_velha = jogo_da_velha.reindex(columns=atributos)
# jogo_da_velha


limiar = 1 # O limiar separa o index do que é atributo para o que é a classe
atributos = jogo_da_velha.columns[limiar:] # seleciono apenas os atributos

# Agora é possivel aplicar o StandardScaler apenas nos atributos

jogo_da_velha[atributos] = StandardScaler().fit_transform(jogo_da_velha[atributos])
jogo_da_velha[atributos]

## Aprendizagem supervisionada
##### Selecionando o X e y

In [ ]:
# Selecionando o X e o y


# X = jogo_da_velha[atributos]
X = jogo_da_velha.iloc[:, limiar:] # Atributos do Data Frame do Pandas
y = jogo_da_velha.iloc[:, :limiar] # Classes do Data Frame do Pandas

y = np.squeeze(y) # Converter em um unico Array quando usar o dataFrame do Pandas

y_unique = np.unique(y)

from sklearn.preprocessing import LabelEncoder # Criando um mapa de cores para mapear o gráfico

lb = LabelEncoder()
lb.fit(y)
mapa_cores = lb.transform(y)

y_unique # Valores unicos de classes

##### PCA

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print('Projecting %d-dimensional data to 2D' % X.shape[1])

def plot_g(pyplot, title, colors, n_classes):
    pyplot.scatter(X_reduced[:, 0], X_reduced[:, 1], c=colors, 
             edgecolor='none', alpha=0.7, s=40,
             cmap=plt.cm.get_cmap('nipy_spectral', n_classes)
           )
    pyplot.colorbar()
    pyplot.title(title)

plt.figure(figsize=(10, 8)) # Tamanho da Figura
plot_g(plt, 'Projeção 2D do Dados em PCA', mapa_cores, len(y_unique))
plt.show()


##### Plotando os Gráficos 

In [ ]:

# plt.figure(figsize=(10*2, 7))

# plt.subplot(1, 2, 1) # row 1, col 2 index 1
# plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels, 
#             edgecolor='none', alpha=0.7, s=40,
#             cmap=plt.cm.get_cmap('nipy_spectral', len(y_unique)))
# plt.colorbar()
# plt.title('Projeção KMeans com X Original')

# plt.subplot(1, 2, 2) # row 1, col 2 index 1
# plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=labels_PCA, 
#             edgecolor='none', alpha=0.7, s=40,
#             cmap=plt.cm.get_cmap('nipy_spectral', len(y_unique)))
# plt.colorbar()
# plt.title('Projeção KMeans com X PCA')

# plt.show()

# Utilização do k-NN

##### Sem PCA

In [ ]:
dataset = jogo_da_velha

i_sort = np.random.randint(len(X) - 1) # Selecionando aleatóriamente um index de teste

X_teste = X.iloc[i_sort] 
X = X.drop(i_sort)

y_teste = y.iloc[i_sort]
y = y.drop(i_sort)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X, y)

predict = knn.predict(np.array(X_teste).reshape(1,-1))

print({"Previsto:": predict, "Correto:": y_teste})

In [ ]:
dist, vizinhos = knn.kneighbors(np.array(X_teste).reshape(1,-1))
classes = []
for v in vizinhos:
    classes.append(dataset.iloc[:-1, limiar][v])
classes